# Podstawowe transformacje SQL i PySpark

**Cel szkoleniowy:** Opanowanie podstawowych transformacji danych w PySpark DataFrame API i SQL, zrozumienie ekwiwalentności obu podejść, umiejętność wyboru odpowiedniego narzędzia do zadania

**Zakres tematyczny:**
- Transformacje kolumnowe: select(), withColumn(), drop(), alias()
- Logika warunkowa: when() / otherwise() oraz CASE WHEN
- Operacje tekstowe: regexp_replace(), trim(), lower(), upper()
- Filtry i sortowanie: filter(), where(), orderBy()
- Agregacje: groupBy(), agg(), rollup(), cube()
- SQL equivalents i porównanie wydajności

## Kontekst i wymagania

- **Dzień szkolenia**: Dzień 1 - Fundamentals & Exploration
- **Typ notebooka**: Demo
- **Wymagania techniczne**:
  - Databricks Runtime 13.0+ (zalecane: 14.3 LTS)
  - Unity Catalog włączony
  - Uprawnienia: CREATE TABLE, CREATE SCHEMA, SELECT, MODIFY
  - Klaster: Standard z 2-4 workers
- **Czas trwania**: 60 minut
- **Prerekvizity**: 02_data_import_exploration.ipynb

## Wstęp teoretyczny

**Cel sekcji:** Zrozumienie fundamentów transformacji danych w Spark i związku między DataFrame API a SQL

**Podstawowe pojęcia:**
- **DataFrame API (PySpark)**: Programatyczne API do manipulacji danymi używające metod i funkcji Pythona
- **Spark SQL**: Deklaratywny język zapytań SQL na DataFrames
- **Catalyst Optimizer**: Silnik optymalizacji Spark, który kompiluje zarówno DataFrame API jak i SQL do tego samego execution plan
- **Lazy Evaluation**: Transformacje nie są wykonywane natychmiast, tylko budują DAG (Directed Acyclic Graph) wykonywany przy akcji
- **Transformations vs Actions**: Transformacje (select, filter, groupBy) są lazy, akcje (show, count, collect) triggerują wykonanie

**Dlaczego to ważne?**
Spark oferuje dwa równoważne sposoby transformacji danych: DataFrame API (PySpark) i SQL. Oba kompilują się do tego samego execution plan przez Catalyst Optimizer, więc wydajność jest identyczna. Wybór zależy od preferencji zespołu, złożoności logiki i integracji z innymi narzędziami. Znajomość obu podejść jest kluczowa dla Data Engineer.

## Izolacja per użytkownik

Uruchom skrypt inicjalizacyjny dla per-user izolacji katalogów i schematów:

In [0]:
%run ../00_setup

## Konfiguracja

Import bibliotek i ustawienie zmiennych środowiskowych:

In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
import re
from datetime import datetime, timedelta

### Kontekst użytkownika

Wyświetlimy konfigurację środowiska z 00_setup:

In [0]:
print("=== Kontekst użytkownika ===")
print(f"Katalog: {CATALOG}")
print(f"Schema Bronze: {BRONZE_SCHEMA}")
print(f"Schema Silver: {SILVER_SCHEMA}")
print(f"Schema Gold: {GOLD_SCHEMA}")
print(f"Użytkownik: {raw_user}")

### Konfiguracja katalogu domyślnego

Ustawienie katalogu i schematu jako domyślne:

In [0]:
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE SCHEMA {BRONZE_SCHEMA}")

## Przygotowanie danych

**Wprowadzenie teoretyczne:**

Załadujemy dane z Unity Catalog Volume - bezpiecznego, zarządzanego storage dla plików. Użyjemy danych customers i orders do demonstracji transformacji.

**Kluczowe pojęcia:**
- **Volume**: Kontener dla plików w Unity Catalog z kontrolą dostępu
- **createOrReplaceTempView**: Rejestracja DataFrame jako temporary view dla zapytań SQL
- **Temp View**: Widok tymczasowy dostępny tylko w bieżącej sesji Spark

**Zastosowanie praktyczne:**
- Ładowanie danych źródłowych z managed storage
- Przygotowanie danych do transformacji
- Umożliwienie zapytań SQL na DataFrames

### Wczytanie danych z Volume

**Cel:** Załadowanie danych customers i orders z Unity Catalog Volume

**Podejście:**
1. Zdefiniowanie ścieżek do plików w Volume
2. Wczytanie CSV i JSON
3. Rejestracja jako temporary views dla SQL

In [0]:
# RESOURCE: CSV i JSON files w volume
# VARIABLE: df_customers, df_orders - DataFrames z danymi

# Ścieżki do plików w Volume
customers_path = f"{DATASET_BASE_PATH}/customers/customers.csv"
orders_path = f"{DATASET_BASE_PATH}/orders/orders_batch.json"

# Wczytanie customers (CSV)
df_customers = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(customers_path)

# Wczytanie orders (JSON)
df_orders = spark.read \
    .format("json") \
    .option("inferSchema", "true") \
    .load(orders_path)

# Rejestracja jako temporary views dla SQL queries
df_customers.createOrReplaceTempView("customers")
df_orders.createOrReplaceTempView("orders")


### Wczytanie pliku customers (CSV)

Załadowanie danych customers z Volume:

In [0]:
df_customers = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(customers_path)

### Wczytanie pliku orders (JSON)

Załadowanie danych orders z Volume:

In [0]:
df_orders = spark.read \
    .format("json") \
    .option("inferSchema", "true") \
    .load(orders_path)

### Rejestracja temporary views

Utworzenie temporary views dla zapytań SQL:

In [0]:
df_customers.createOrReplaceTempView("customers")
df_orders.createOrReplaceTempView("orders")

### Podgląd danych orders

Sprawdzenie pierwszych 5 rekordów orders:

In [0]:
display(spark.table("orders").limit(5))

In [0]:
df_orders.printSchema()

**Schemat danych orders:**

Struktura DataFrame orders:

In [0]:
%sql
select * from customers limit 5;

### Podgląd danych customers (SQL)

Sprawdzenie pierwszych 5 rekordów customers używając SQL:

In [0]:
df_customers.printSchema()

**Schemat danych customers:**

Struktura DataFrame customers:

### Podsumowanie załadowanych danych

**Dane zostały pomyślnie załadowane z Unity Catalog Volume:**

- **Customers**: CSV file z informacjami o klientach  
- **Orders**: JSON file z danymi zamówień
- **Temporary views**: Utworzone widoki `customers` i `orders` dla zapytań SQL
- **Schemat**: Automatycznie wykryty dzięki `inferSchema=true`

**Weryfikacja:**
Sprawdź liczbę rekordów i kolumn używając `df.count()` i `len(df.columns)` jeśli potrzebne.

**Wyjaśnienie:**

Załadowaliśmy dane z Volume używając `spark.read` z opcjami specyficznymi dla formatów. `createOrReplaceTempView()` rejestruje DataFrame jako SQL view, co pozwala na zapytania SQL. Views są tymczasowe i dostępne tylko w bieżącej sesji.

## Sekcja 1: Transformacje kolumnowe

**Wprowadzenie teoretyczne:**

Transformacje kolumnowe to najczęstsze operacje w data engineering: selekcja potrzebnych kolumn, dodawanie nowych kolumn obliczeniowych, zmiana nazw i usuwanie zbędnych kolumn. Spark oferuje bogate API do manipulacji kolumnami.

**Kluczowe pojęcia:**
- **select()**: Wybór i projekcja kolumn (PySpark)
- **withColumn()**: Dodanie lub nadpisanie kolumny
- **drop()**: Usunięcie kolumn
- **alias()**: Zmiana nazwy kolumny lub wyrażenia
- **col() / F.col()**: Referencja do kolumny w DataFrame

**Zastosowanie praktyczne:**
- Selekcja tylko potrzebnych kolumn (column pruning dla wydajności)
- Kalkulacje biznesowe (np. total_price = quantity * unit_price)
- Renaming dla czytelności
- Czyszczenie niepotrzebnych kolumn

### Przykład 1.1: Selekcja kolumn - select()

**Cel:** Wybór konkretnych kolumn z DataFrame

**Podejście:**
1. Selekcja po nazwach kolumn
2. Selekcja z aliasami
3. Selekcja z obliczeniami

In [0]:
# RESOURCE: DataFrame df_customers
# VARIABLE: df_selected - DataFrame z wybranymi kolumnami

# PySpark: Selekcja kolumn
df_selected = df_customers.select(
    "customer_id",
    "first_name",
    "last_name",
    "email",
    "city",
    "country"
)

display(df_selected.limit(5))


**Wynik:** DataFrame z wybranymi kolumnami - selekcja tylko potrzebnych pól do analizy.

In [0]:
df_selected_alias = df_customers.select(
    F.col("customer_id").alias("id"),
    F.concat_ws(' ', F.col("first_name"), F.col("last_name")).alias("full_name"),
    F.col("email"),
    F.concat_ws(", ", F.col("city"), F.col("country")).alias("location")
)

display(df_selected_alias.limit(5))

**Wynik:** DataFrame z aliasami i obliczeniami - `concat_ws()` łączy stringi z separatorem, `alias()` nadaje nowe nazwy kolumnom.

### Przykład 1.2: SQL Equivalent - SELECT

**Cel:** Ta sama operacja w SQL

In [0]:
%sql

    SELECT 
        customer_id,
        first_name,
        last_name,
        email,
        city,
        country
    FROM customers


In [0]:
%sql
 SELECT 
        customer_id AS id,
        first_name || ' ' || last_name AS full_name,
        email,
        CONCAT(city, ', ', country) AS location
    FROM customers

### Przykład 1.3: Dodawanie nowych kolumn - withColumn()

**Cel:** Tworzenie nowych kolumn na podstawie istniejących

In [0]:
display(df_customers)

In [0]:
# RESOURCE: DataFrame df_customers
# VARIABLE: df_with_new_cols - DataFrame z nowymi kolumnami

# PySpark: withColumn() - dodawanie nowych kolumn
df_with_new_cols = df_customers \
    .withColumn("full_name", F.upper(F.concat_ws(' ', F.col("first_name"), F.col("last_name")))) \
    .withColumn("email_domain", F.split(F.col("email"), "@").getItem(1)) \
    .withColumn("registration_year", F.year(F.col("registration_date"))) \
    .withColumn("is_premium", F.when(F.col("customer_segment") == "Premium", True).otherwise(False))

display(df_with_new_cols.select(
    "first_name","last_name", "full_name", 
    "email", "email_domain",
    "registration_date", "registration_year",
    "customer_segment", "is_premium"
).limit(5))

### Wyświetlenie wyniku

Podgląd DataFrame z nowymi kolumnami:

In [0]:
display(df_with_new_cols.select(
    "first_name","last_name", "full_name", 
    "email", "email_domain",
    "registration_date", "registration_year",
    "customer_segment", "is_premium"
).limit(5))

### Przykład 1.4: SQL Equivalent - kolumny w SELECT

**Cel:** Tworzenie nowych kolumn w SQL

In [0]:

%sql
-- SQL: Nowe kolumny w SELECT
    SELECT 
        first_name || ' ' || last_name AS full_name,
        UPPER(first_name || ' ' || last_name) AS full_name_upper,
        email,
        -- GET pobiera element o podanym indeksie z tablicy zwróconej przez SPLIT (indeksowanie od 0)
        get(SPLIT(email, '@'), 1) AS email_domain,
        registration_date,
        YEAR(registration_date) AS registration_year,
        customer_segment,
        customer_segment = 'Premium' AS is_premium
    FROM customers



## Null-e w agregacjach – na co uważać?

Przy agregacjach warto pamiętać, że:

- `COUNT(*)` liczy wszystkie wiersze,
- `COUNT(kolumna)` liczy tylko wiersze, gdzie `kolumna IS NOT NULL`,
- `COUNT(DISTINCT kolumna)` liczy unikalne wartości **niepuste**.

### Przykład 1.5: Usuwanie kolumn - drop()

**Cel:** Usunięcie niepotrzebnych kolumn

In [0]:
# RESOURCE: DataFrame df_customers
# VARIABLE: df_dropped - DataFrame z usuniętymi kolumnami

# PySpark: drop() - usuwanie kolumn
df_dropped = df_customers.drop("phone", "postal_code", "notes")

# SQL: Wybór wszystkich kolumn OPRÓCZ niektórych (trzeba wylistować ręcznie)
# W SQL nie ma drop(), więc trzeba wybrać tylko potrzebne kolumny

**Wynik:** DataFrame z usuniętymi kolumnami `phone`, `postal_code`, `notes`

**Weryfikacja:**
- Kolumny przed: 12 kolumn
- Kolumny po: 9 kolumn (usunięto 3)
- **SQL equivalent**: W SQL nie ma drop() - trzeba wybrać tylko potrzebne kolumny w SELECT

In [0]:
display(df_dropped.limit(3))

## Sekcja 2: Logika warunkowa

**Wprowadzenie teoretyczne:**

Logika warunkowa pozwala na tworzenie wartości kolumn bazując na warunkach. Jest fundamentem business logic w transformacjach danych - kategoryzacja, flagowanie, mapowanie wartości.

**Kluczowe pojęcia:**
- **when() / otherwise()**: Konstrukcja IF-THEN-ELSE w PySpark
- **CASE WHEN**: Konstrukcja warunkowa w SQL
- **Chaining conditions**: Łańcuchowe warunki (if-elif-else)
- **Boolean expressions**: Wyrażenia logiczne jako warunki

**Zastosowanie praktyczne:**
- Kategoryzacja klientów (VIP, Standard, New)
- Status mapping (active/inactive)
- Business rules (discounts, flags)
- Data quality flags (valid/invalid)

### Przykład 2.1: Logika warunkowa - when() / otherwise()

**Cel:** Kategoryzacja klientów na podstawie istniejących danych (customer_segment, registration_date)

**Podejście:**
1. Proste when-otherwise (IF-ELSE)
2. Łańcuchowe when (IF-ELIF-ELSE)
3. Multiple conditions

In [0]:
# RESOURCE: DataFrame df_customers
# VARIABLE: df_categorized - DataFrame z kategoriami klientów

# Dodanie sztucznej kolumny account_value na podstawie istniejących danych dla demonstracji
df_customers_with_value = df_customers.withColumn(
    "account_value", 
    F.when(F.col("customer_segment") == "Premium", F.rand() * 10000 + 5000)
     .when(F.col("customer_segment") == "Standard", F.rand() * 5000 + 1000)
     .otherwise(F.rand() * 1000 + 100)
)

# PySpark: when() / otherwise() - logika warunkowa
df_categorized = df_customers_with_value.withColumn(
    "customer_tier",
    F.when(F.col("account_value") >= 8000, "Platinum")
     .when(F.col("account_value") >= 3000, "Gold")
     .when(F.col("account_value") >= 1000, "Silver")
     .otherwise("Bronze")
)

# Dodanie flagi VIP (multiple conditions)
df_categorized = df_categorized.withColumn(
    "is_vip",
    F.when(
        (F.col("account_value") > 5000) & 
        (F.col("customer_segment") == "Premium"),
        True
    ).otherwise(False)
)

# Kategoryzacja na podstawie daty rejestracji
df_categorized = df_categorized.withColumn(
    "registration_period",
    F.when(F.col("registration_date") >= "2025-01-01", "Recent")
     .when(F.col("registration_date") >= "2024-01-01", "This Year")
     .otherwise("Older")
)

print("=== PySpark: when() / otherwise() ===")
display(df_categorized.select(
    "first_name", "last_name",
    "customer_segment", "account_value", "customer_tier",
    "is_vip", "registration_date", "registration_period"
).orderBy(F.desc("account_value")).limit(10))

### Kategoryzacja customer_tier

Tworzenie kategorii klientów na podstawie account_value:

In [0]:
df_categorized = df_customers_with_value.withColumn(
    "customer_tier",
    F.when(F.col("account_value") >= 8000, "Platinum")
     .when(F.col("account_value") >= 3000, "Gold")
     .when(F.col("account_value") >= 1000, "Silver")
     .otherwise("Bronze")
)

### Flaga VIP (multiple conditions)

Dodanie flagi VIP na podstawie wielu warunków:

In [0]:
df_categorized = df_categorized.withColumn(
    "is_vip",
    F.when(
        (F.col("account_value") > 5000) & 
        (F.col("customer_segment") == "Premium"),
        True
    ).otherwise(False)
)

### Kategoryzacja na podstawie daty rejestracji

Dodanie kategorii na podstawie registration_date:

In [0]:
df_categorized = df_categorized.withColumn(
    "registration_period",
    F.when(F.col("registration_date") >= "2025-01-01", "Recent")
     .when(F.col("registration_date") >= "2024-01-01", "This Year")
     .otherwise("Older")
)

### Wyświetlenie wyników logiki warunkowej

Podgląd DataFrame z wszystkimi kategoriami (posortowany według account_value):

In [0]:
display(df_categorized.select(
    "first_name", "last_name",
    "customer_segment", "account_value", "customer_tier",
    "is_vip", "registration_date", "registration_period"
).orderBy(F.desc("account_value")).limit(10))

### Przykład 2.2: SQL Equivalent - CASE WHEN

**Cel:** Ta sama logika warunkowa w SQL

In [0]:
# Najpierw utwórz temporary view z account_value
df_customers_with_value.createOrReplaceTempView("customers_enriched")

# SQL: CASE WHEN - logika warunkowa
df_categorized_sql = spark.sql("""
    SELECT 
        first_name,
        last_name,
        customer_segment,
        account_value,
        CASE 
            WHEN account_value >= 8000 THEN 'Platinum'
            WHEN account_value >= 3000 THEN 'Gold'
            WHEN account_value >= 1000 THEN 'Silver'
            ELSE 'Bronze'
        END AS customer_tier,
        CASE 
            WHEN account_value > 5000 AND customer_segment = 'Premium' THEN TRUE
            ELSE FALSE
        END AS is_vip,
        registration_date,
        CASE 
            WHEN registration_date >= '2025-01-01' THEN 'Recent'
            WHEN registration_date >= '2024-01-01' THEN 'This Year'
            ELSE 'Older'
        END AS registration_period
    FROM customers_enriched
    ORDER BY account_value DESC
""")

print("=== SQL: CASE WHEN ===")
display(df_categorized_sql.limit(10))

## Sekcja 3: Operacje tekstowe

**Wprowadzenie teoretyczne:**

Operacje tekstowe są kluczowe w data cleaning i standaryzacji. Spark oferuje bogate API do manipulacji stringami - od prostych (trim, upper) po złożone (regex, split, concat).

**Kluczowe pojęcia:**
- **trim() / ltrim() / rtrim()**: Usuwanie whitespace
- **upper() / lower() / initcap()**: Zmiana wielkości liter
- **regexp_replace()**: Zastępowanie wzorców regex
- **concat() / concat_ws()**: Łączenie stringów
- **split() / substring()**: Dzielenie i wycinanie stringów

**Zastosowanie praktyczne:**
- Data cleaning (whitespace, case normalization)
- Parsing (extracting parts of strings)
- Formatting (phone numbers, codes)
- Anonymization (masking sensitive data)

### Przykład 3.1: Operacje tekstowe - trim, upper, lower

**Cel:** Czyszczenie i normalizacja pól tekstowych

In [0]:
# RESOURCE: DataFrame df_customers
# VARIABLE: df_cleaned_text - DataFrame z oczyszczonymi polami tekstowymi

# PySpark: Operacje tekstowe
df_cleaned_text = df_customers \
    .withColumn("first_name_clean", F.trim(F.col("first_name"))) \
    .withColumn("last_name_clean", F.trim(F.col("last_name"))) \
    .withColumn("full_name_upper", F.upper(F.concat_ws(' ', F.trim(F.col("first_name")), F.trim(F.col("last_name"))))) \
    .withColumn("full_name_title", F.initcap(F.concat_ws(' ', F.trim(F.col("first_name")), F.trim(F.col("last_name"))))) \
    .withColumn("email_lower", F.lower(F.trim(F.col("email")))) \
    .withColumn("country_code", F.upper(F.col("country")))

display(df_cleaned_text.select(
    "first_name", "last_name", "first_name_clean", "last_name_clean", 
    "full_name_upper", "full_name_title",
    "email", "email_lower",
    "country", "country_code"
).limit(5))

**Wynik:** DataFrame z oczyszczonymi polami tekstowymi

**Zastosowane funkcje:**
- `trim()`: Usuwa whitespace z początku i końca
- `upper()`, `lower()`: Zmiana wielkości liter  
- `initcap()`: Pierwsza litera wielka (Title Case)
- `concat_ws()`: Łączenie stringów z separatorem

### Przykład 3.2: SQL Equivalent - funkcje tekstowe

**Cel:** Te same operacje w SQL

In [0]:
# SQL: Funkcje tekstowe
df_cleaned_text_sql = spark.sql("""
    SELECT 
        first_name,
        last_name,
        TRIM(first_name) AS first_name_clean,
        TRIM(last_name) AS last_name_clean,
        UPPER(TRIM(first_name) || ' ' || TRIM(last_name)) AS full_name_upper,
        INITCAP(TRIM(first_name) || ' ' || TRIM(last_name)) AS full_name_title,
        email,
        LOWER(TRIM(email)) AS email_lower,
        country,
        UPPER(country) AS country_code
    FROM customers
""")

display(df_cleaned_text_sql.limit(5))

### Przykład 3.3: Regex i zaawansowane operacje

**Cel:** Używanie regex do czyszczenia i parsowania

In [0]:
# RESOURCE: DataFrame df_customers
# VARIABLE: df_regex - DataFrame z operacjami regex

# PySpark: regexp_replace, split, substring
df_regex = df_customers \
    .withColumn("phone_digits_only", 
        F.regexp_replace(F.col("phone"), "[^0-9]", "")
    ) \
    .withColumn("email_username",
        F.split(F.col("email"), "@").getItem(0)
    ) \
    .withColumn("email_domain",
        F.split(F.col("email"), "@").getItem(1)
    ) \
    .withColumn("customer_initials",
        F.concat(
            F.substring(F.col("first_name"), 1, 1),
            F.lit("."),
            F.substring(F.col("last_name"), 1, 1),
            F.lit(".")
        )
    )

display(df_regex.select(
    "phone", "phone_digits_only",
    "email", "email_username", "email_domain",
    "first_name", "last_name", "customer_initials"
).limit(5))

### Przykład 3.4: SQL Equivalent - regex i parsing

**Cel:** Te same operacje regex w SQL

In [0]:
# SQL: Regex i funkcje tekstowe
df_regex_sql = spark.sql("""
    SELECT 
        phone,
        REGEXP_REPLACE(phone, '[^0-9]', '') AS phone_digits_only,
        email,
        SPLIT(email, '@')[0] AS email_username,
        SPLIT(email, '@')[1] AS email_domain,
        first_name,
        last_name,
        CONCAT(
            SUBSTRING(first_name, 1, 1), '.', 
            SUBSTRING(last_name, 1, 1), '.'
        ) AS customer_initials
    FROM customers
""")

display(df_regex_sql.limit(5))

## Sekcja 4: Filtry i sortowanie

**Wprowadzenie teoretyczne:**

Filtry redukują zbiór danych do interesujących nas rekordów, a sortowanie organizuje wyniki. Są to fundamentalne operacje analityczne, często łączone z agregacjami.

**Kluczowe pojęcia:**
- **filter() / where()**: Filtrowanie rekordów (synonimy)
- **orderBy() / sort()**: Sortowanie (synonimy)
- **Boolean conditions**: Warunki logiczne (AND, OR, NOT)
- **Multiple conditions**: Łączenie wielu warunków
- **asc() / desc()**: Kierunek sortowania

**Zastosowanie praktyczne:**
- Filtrowanie do interesujących rekordów (active customers, recent orders)
-排序 dla analiz (top customers, oldest records)
- Pre-aggregation filtering (WHERE przed GROUP BY)
- Post-aggregation filtering (HAVING po GROUP BY)

### Przykład 4.1: Filtry - filter() / where()

**Cel:** Filtrowanie rekordów na podstawie warunków

**Podejście:**
1. Proste filtry
2. Multiple conditions (AND, OR)
3. Filtry na wartościach null

In [0]:
# RESOURCE: DataFrame df_customers
# VARIABLE: df_filtered - DataFrame z przefiltrowanymi rekordami

# PySpark: filter() / where() - identyczne
df_filtered_premium = df_customers.filter(F.col("customer_segment") == "Premium")

print("=== PySpark: filter() - Premium customers ===")
print(f"Total customers: {df_customers.count()}")
print(f"Premium customers: {df_filtered_premium.count()}")
display(df_filtered_premium.limit(5))

# Multiple conditions z AND (&)
df_filtered_multi = df_customers.filter(
    (F.col("customer_segment") == "Premium") & 
    (F.col("country") == "USA") &
    (F.col("state").isNotNull())
)

print("\n=== PySpark: Multiple conditions (AND) ===")
print(f"Premium USA customers with state info: {df_filtered_multi.count()}")
display(df_filtered_multi.limit(5))

# OR conditions (|)
df_filtered_or = df_customers.filter(
    (F.col("country") == "USA") | 
    (F.col("country") == "Poland")
)

print("\n=== PySpark: OR conditions ===")
print(f"Customers from USA or Poland: {df_filtered_or.count()}")

# NOT condition (~) i null check
df_filtered_not_null = df_customers.filter(
    F.col("email").isNotNull() &
    ~F.col("email").contains("example.com")
)

print("\n=== PySpark: NOT i null check ===")
print(f"Customers with valid email (not null, not example.com): {df_filtered_not_null.count()}")

**Wynik:** Filtr Premium customers

Porównanie: Total customers vs Premium customers

In [0]:
display(df_filtered_premium.limit(5))

### Multiple conditions (AND)

Filtr z wieloma warunkami połączonymi operatorem AND (&):

In [0]:
df_filtered_multi = df_customers.filter(
    (F.col("customer_segment") == "Premium") & 
    (F.col("country") == "USA") &
    (F.col("state").isNotNull())
)

display(df_filtered_multi.limit(5))

### Przykład 4.2: SQL Equivalent - WHERE

**Cel:** Te same filtry w SQL

In [0]:
# SQL: WHERE clause
df_filtered_premium_sql = spark.sql("""
    SELECT * FROM customers
    WHERE customer_segment = 'Premium'
""")

print("=== SQL: WHERE - Premium customers ===")
print(f"Premium customers: {df_filtered_premium_sql.count()}")
display(df_filtered_premium_sql.limit(5))

# Multiple conditions AND
df_filtered_multi_sql = spark.sql("""
    SELECT * FROM customers
    WHERE customer_segment = 'Premium'
        AND country = 'USA'
        AND state IS NOT NULL
""")

print("\n=== SQL: Multiple conditions (AND) ===")
print(f"Count: {df_filtered_multi_sql.count()}")

# OR conditions
df_filtered_or_sql = spark.sql("""
    SELECT * FROM customers
    WHERE country = 'USA' 
       OR country = 'Poland'
""")

print("\n=== SQL: OR conditions ===")
print(f"Count: {df_filtered_or_sql.count()}")

# NOT i null check
df_filtered_not_null_sql = spark.sql("""
    SELECT * FROM customers
    WHERE email IS NOT NULL
        AND email NOT LIKE '%example.com%'
""")

print("\n=== SQL: NOT i null check ===")
print(f"Count: {df_filtered_not_null_sql.count()}")

### Przykład 4.3: Sortowanie - orderBy() / sort()

**Cel:** Sortowanie wyników

In [0]:
# RESOURCE: DataFrame df_customers
# VARIABLE: df_sorted - DataFrame posortowany

# PySpark: orderBy() / sort() - identyczne
df_sorted_desc = df_customers.orderBy(F.desc("registration_date"))

display(df_sorted_desc.select("first_name", "last_name", "registration_date", "country").limit(10))

# Multiple columns sort
df_sorted_multi = df_customers.orderBy(
    F.col("country").asc(),
    F.col("customer_segment").desc(),
    F.col("registration_date").desc()
)

print("\n=== PySpark: Multiple columns sort (country ASC, segment DESC, date DESC) ===")
display(df_sorted_multi.select("first_name", "last_name", "country", "customer_segment", "registration_date").limit(10))

# Null handling w sortowaniu
df_sorted_nulls = df_customers.orderBy(
    F.col("email").asc_nulls_last()
)

print("\n=== PySpark: Sort with nulls handling (nulls last) ===")
display(df_sorted_nulls.select("first_name", "last_name", "email").limit(10))

### Multiple columns sort

Sortowanie po wielu kolumnach z różnymi kierunkami:

In [0]:
df_sorted_multi = df_customers.orderBy(
    F.col("country").asc(),
    F.col("customer_segment").desc(),
    F.col("registration_date").desc()
)

display(df_sorted_multi.select("first_name", "last_name", "country", "customer_segment", "registration_date").limit(10))

### Przykład 4.4: SQL Equivalent - ORDER BY

**Cel:** Te same sortowania w SQL

In [0]:
# SQL: ORDER BY
df_sorted_desc_sql = spark.sql("""
    SELECT first_name, last_name, registration_date, country
    FROM customers
    ORDER BY registration_date DESC
""")

print("=== SQL: ORDER BY DESC ===")
display(df_sorted_desc_sql.limit(10))

# Multiple columns
df_sorted_multi_sql = spark.sql("""
    SELECT first_name, last_name, country, customer_segment, registration_date
    FROM customers
    ORDER BY country ASC, customer_segment DESC, registration_date DESC
""")

print("\n=== SQL: Multiple columns ORDER BY ===")
display(df_sorted_multi_sql.limit(10))

# Nulls handling
df_sorted_nulls_sql = spark.sql("""
    SELECT first_name, last_name, email
    FROM customers
    ORDER BY email ASC NULLS LAST
""")

print("\n=== SQL: ORDER BY with NULLS LAST ===")
display(df_sorted_nulls_sql.limit(10))

## Sekcja 5: Agregacje

**Wprowadzenie teoretyczne:**

Agregacje redukują wiele rekordów do wartości podsumowujących. Są fundamentem analityki biznesowej - KPIs, metryki, raporty. Spark oferuje standardowe agregacje (sum, avg, count) oraz zaawansowane (rollup, cube).

**Kluczowe pojęcia:**
- **groupBy()**: Grupowanie po kolumnach
- **agg()**: Agregacje (sum, avg, count, min, max)
- **rollup()**: Hierarchiczne podsumowania (subtotals)
- **cube()**: Wszystkie kombinacje wymiarów (cross-tabulation)
- **HAVING**: Filtrowanie wyników agregacji

**Zastosowanie praktyczne:**
- KPIs per country, per customer tier
- Sales reports (total revenue, average order value)
- Cohort analysis (customers per age group)
- Multi-dimensional reporting (rollup, cube)

### Przykład 5.1: Podstawowe agregacje - groupBy() + agg()

**Cel:** Obliczenie statystyk per grupa

**Podejście:**
1. Proste groupBy + count
2. Multiple aggregations
3. Aliasy dla czytelności

In [0]:
# RESOURCE: DataFrame df_customers
# VARIABLE: df_agg - DataFrame z agregacjami

# Dodajmy kolumnę account_value na podstawie customer_segment dla demonstracji agregacji
df_customers_for_agg = df_customers.withColumn(
    "account_value",
    F.when(F.col("customer_segment") == "Premium", F.rand() * 5000 + 5000)
     .when(F.col("customer_segment") == "Standard", F.rand() * 3000 + 2000)
     .otherwise(F.rand() * 1000 + 500)
)

# PySpark: groupBy() + agg()
df_agg_country = df_customers_for_agg.groupBy("country").agg(
    F.count("*").alias("customer_count"),
    F.sum("account_value").alias("total_account_value"),
    F.avg("account_value").alias("avg_account_value"),
    F.max("account_value").alias("max_account_value"),
    F.count(F.when(F.col("customer_segment") == "Premium", 1)).alias("premium_count")
).orderBy(F.desc("customer_count"))

print("=== PySpark: groupBy() + agg() per country ===")
display(df_agg_country)

# Multiple dimensions grouping
df_agg_multi = df_customers_for_agg \
    .withColumn("registration_year", F.year(F.col("registration_date"))) \
    .groupBy("country", "customer_segment") \
    .agg(
        F.count("*").alias("count"),
        F.avg("account_value").alias("avg_account_value")
    ) \
    .orderBy("country", "customer_segment")

print("\n=== PySpark: groupBy() multiple dimensions ===")
display(df_agg_multi)

### Agregacja per country

Obliczenie statystyk per kraj:

In [0]:
df_agg_country = df_customers_for_agg.groupBy("country").agg(
    F.count("*").alias("customer_count"),
    F.sum("account_value").alias("total_account_value"),
    F.avg("account_value").alias("avg_account_value"),
    F.max("account_value").alias("max_account_value"),
    F.count(F.when(F.col("customer_segment") == "Premium", 1)).alias("premium_count")
).orderBy(F.desc("customer_count"))

display(df_agg_country)

### Multiple dimensions grouping

Grupowanie po wielu wymiarach (country + customer_segment):

In [0]:
df_agg_multi = df_customers_for_agg \
    .withColumn("registration_year", F.year(F.col("registration_date"))) \
    .groupBy("country", "customer_segment") \
    .agg(
        F.count("*").alias("count"),
        F.avg("account_value").alias("avg_account_value")
    ) \
    .orderBy("country", "customer_segment")

display(df_agg_multi)

### Przykład 5.2: SQL Equivalent - GROUP BY

**Cel:** Te same agregacje w SQL

In [0]:
df_customers_for_agg.createOrReplaceTempView("customers_with_value")

In [0]:
# SQL: GROUP BY
df_agg_country_sql = spark.sql("""
    SELECT 
        country,
        COUNT(*) AS customer_count,
        SUM(account_value) AS total_account_value,
        AVG(account_value) AS avg_account_value,
        MAX(account_value) AS max_account_value,
        COUNT(CASE WHEN customer_segment = 'Premium' THEN 1 END) AS premium_count
    FROM customers_with_value
    GROUP BY country
    ORDER BY customer_count DESC
""")

print("=== SQL: GROUP BY per country ===")
display(df_agg_country_sql)

# Multiple dimensions
df_agg_multi_sql = spark.sql("""
    SELECT 
        country,
        customer_segment,
        COUNT(*) AS count,
        AVG(account_value) AS avg_account_value
    FROM customers_with_value
    GROUP BY country, customer_segment
    ORDER BY country, customer_segment
""")

print("\n=== SQL: GROUP BY multiple dimensions ===")
display(df_agg_multi_sql)

### Przykład 5.3: HAVING - filtrowanie wyników agregacji

**Cel:** Filtrowanie grup po agregacji

In [0]:
# RESOURCE: DataFrame df_customers_for_agg
# VARIABLE: df_having - DataFrame z filtrem po agregacji

# PySpark: HAVING equivalent - filter po agg
df_having = df_customers_for_agg.groupBy("country").agg(
    F.count("*").alias("customer_count"),
    F.avg("account_value").alias("avg_account_value")
).filter(F.col("customer_count") >= 50).orderBy(F.desc("avg_account_value"))

display(df_having)

### Przykład 5.4: SQL Equivalent - HAVING

**Cel:** HAVING clause w SQL

In [0]:
# SQL: HAVING
df_having_sql = spark.sql("""
    SELECT 
        country,
        COUNT(*) AS customer_count,
        AVG(account_value) AS avg_account_value
    FROM customers_with_value
    GROUP BY country
    HAVING COUNT(*) >= 50
    ORDER BY avg_account_value DESC
""")

display(df_having_sql)

### Przykład 5.5: Zaawansowane agregacje - rollup() i cube()

**Cel:** Hierarchiczne podsumowania i cross-tabulation

**Wyjaśnienie:**
- **rollup(A, B)**: Tworzy subtotals hierarchicznie: (A,B), (A), ()
- **cube(A, B)**: Tworzy wszystkie kombinacje: (A,B), (A), (B), ()

In [0]:
# RESOURCE: DataFrame df_customers_for_agg
# VARIABLE: df_rollup, df_cube - DataFrames z zaawansowanymi agregacjami

# PySpark: rollup() - hierarchiczne subtotals
df_rollup = df_customers_for_agg \
    .withColumn("registration_year", F.year(F.col("registration_date"))) \
    .rollup("country", "customer_segment") \
    .agg(
        F.count("*").alias("count"),
        F.sum("account_value").alias("total_account_value")
    ) \
    .orderBy(F.col("country").asc_nulls_last(), F.col("customer_segment").asc_nulls_last())

display(df_rollup)

# PySpark: cube() - wszystkie kombinacje wymiarów
df_cube = df_customers_for_agg \
    .withColumn("registration_year", F.year(F.col("registration_date"))) \
    .cube("country", "customer_segment") \
    .agg(
        F.count("*").alias("count"),
        F.sum("account_value").alias("total_account_value")
    ) \
    .orderBy(F.col("country").asc_nulls_last(), F.col("customer_segment").asc_nulls_last())

print("\n=== PySpark: cube() - wszystkie kombinacje ===")
print("Zawiera: (country, customer_segment), (country), (customer_segment), (grand_total)")
display(df_cube)

**Wyjaśnienie ROLLUP:** Tworzy hierarchiczne subtotals: (country, customer_segment), (country), (grand_total)

### CUBE() - wszystkie kombinacje

Cube tworzy wszystkie możliwe kombinacje wymiarów:

In [0]:
df_cube = df_customers_for_agg \
    .withColumn("registration_year", F.year(F.col("registration_date"))) \
    .cube("country", "customer_segment") \
    .agg(
        F.count("*").alias("count"),
        F.sum("account_value").alias("total_account_value")
    ) \
    .orderBy(F.col("country").asc_nulls_last(), F.col("customer_segment").asc_nulls_last())

display(df_cube)

**Wyjaśnienie CUBE:** Zawiera wszystkie kombinacje: (country, customer_segment), (country), (customer_segment), (grand_total)

### Przykład 5.6: SQL Equivalent - ROLLUP i CUBE

**Cel:** Te same operacje w SQL

In [0]:
# SQL: ROLLUP
df_rollup_sql = spark.sql("""
    SELECT 
        country,
        customer_segment,
        COUNT(*) AS count,
        SUM(account_value) AS total_account_value
    FROM customers_with_value
    GROUP BY ROLLUP(country, customer_segment)
    ORDER BY country ASC NULLS LAST, customer_segment ASC NULLS LAST
""")

display(df_rollup_sql)

# SQL: CUBE
df_cube_sql = spark.sql("""
    SELECT 
        country,
        customer_segment,
        COUNT(*) AS count,
        SUM(account_value) AS total_account_value
    FROM customers_with_value
    GROUP BY CUBE(country, customer_segment)
    ORDER BY country ASC NULLS LAST, customer_segment ASC NULLS LAST
""")

print("\n=== SQL: CUBE ===")
display(df_cube_sql)

### SQL CUBE

Cube w SQL - wszystkie kombinacje wymiarów:

In [0]:
# SQL: CUBE
df_cube_sql = spark.sql("""
    SELECT 
        country,
        customer_segment,
        COUNT(*) AS count,
        SUM(account_value) AS total_account_value
    FROM customers_with_value
    GROUP BY CUBE(country, customer_segment)
    ORDER BY country ASC NULLS LAST, customer_segment ASC NULLS LAST
""")

display(df_cube_sql)

## Porównanie PySpark vs SQL - Podsumowanie

**DataFrame API (PySpark):**

**Zalety:**
- Programatyczne API - łatwa integracja z Python code
- Type safety (PyCharm autocomplete, type hints)
- Łatwe łańcuchowanie transformacji (method chaining)
- Lepsza dla dynamicznych pipeline'ów (parametryzacja, loops)
- Integracja z Python libraries (pandas, numpy)

**Wady:**
- Bardziej verbose dla prostych zapytań
- Wymaga znajomości PySpark API
- Trudniejsze dla bardzo złożonych joinów

**Kiedy używać PySpark:**
- Złożone pipeline'y ETL z logiką biznesową
- Integracja z ML (MLlib, scikit-learn)
- Dynamiczne transformacje (parametryzowane, warunkowe)
- Gdy zespół preferuje programatyczny styl

## Porównanie PySpark vs SQL - Kompleksowe przykłady

**DataFrame API (PySpark):**

In [0]:
# Przykład: Kompleksowe czyszczenie danych w PySpark

df_cleaned_pyspark = df_customers \
    .fillna({"city": "Unknown", "country": "Unknown"}) \
    .dropna(subset=["customer_id"]) \
    .withColumn("full_name", F.trim(F.initcap(F.concat_ws(' ', F.col("first_name"), F.col("last_name"))))) \
    .withColumn("email", F.lower(F.trim(F.col("email")))) \
    .dropDuplicates(["customer_id"]) \
    .withColumn("registration_year", F.year(F.col("registration_date"))) \
    .filter(F.col("registration_year") >= 2020)

display(df_cleaned_pyspark.select("customer_id", "full_name", "email", "customer_segment", "registration_year").limit(5))

**SQL Equivalent:**

In [0]:
# Najpierw utwórz temporary view
df_customers.createOrReplaceTempView("customers_raw")

# SQL approach
df_cleaned_sql = spark.sql("""
SELECT DISTINCT
    customer_id,
    TRIM(INITCAP(first_name || ' ' || last_name)) as full_name,
    LOWER(TRIM(email)) as email,
    customer_segment,
    YEAR(registration_date) as registration_year,
    COALESCE(city, 'Unknown') as city,
    COALESCE(country, 'Unknown') as country,
    phone,
    registration_date
FROM customers_raw
WHERE customer_id IS NOT NULL
    AND YEAR(registration_date) >= 2020
""")

display(df_cleaned_sql.select("customer_id", "full_name", "email", "customer_segment", "registration_year").limit(5))

**SQL Equivalent:**

**Zalety:**
- Deklaratywny - zwięzły i czytelny
- Standardowy język - znany analitykom biznesowym
- Świetny dla ad-hoc analiz
- Czytelniejszy dla złożonych joinów i subqueries
- Łatwy do debugowania (explain plan)

**Wady:**
- Mniej elastyczny dla dynamicznych pipeline'ów
- Trudniejsza parametryzacja
- Słabsza integracja z Python ecosystem
- String interpolation może być ryzykowna (SQL injection risk)

**Kiedy używać SQL:**
- Ad-hoc analityka i eksploracja
- Raportowanie i dashboardy
- Gdy zespół preferuje SQL
- Proste do średnio złożone transformacje
- Widoki i materialized views

**Porównanie:**
- **Wydajność**: Identyczna - Catalyst optimizer kompiluje oba do tego samego execution plan
- **Best practice**: Używaj tego, co jest bardziej czytelne dla konkretnego przypadku
- **Hybrydowe podejście**: Kombinuj oba (DataFrame API dla pipeline, SQL dla final aggregations)
- **Zespół**: Wybór zależy od umiejętności zespołu (Data Engineers → PySpark, Analysts → SQL)

## Walidacja i weryfikacja

### Checklist - Co powinieneś uzyskać:
- [ ] Dane załadowane z Volume (customers, orders)
- [ ] Temporary views utworzone dla SQL queries
- [ ] Transformacje kolumnowe działają (select, withColumn, drop, alias)
- [ ] Logika warunkowa działa (when/otherwise, CASE WHEN)
- [ ] Operacje tekstowe działają (trim, upper, lower, regex)
- [ ] Filtry i sortowanie działają (filter, orderBy)
- [ ] Agregacje działają (groupBy, agg, rollup, cube)
- [ ] SQL equivalents dają identyczne wyniki co PySpark

### Komendy weryfikacyjne:

In [0]:
# Weryfikacja wyników

print("=== WERYFIKACJA WYNIKÓW ===\n")

# 1. Sprawdź czy temporary views istnieją
print("1. Temporary views:")
temp_views = spark.catalog.listTables()
for view in temp_views:
    if view.name in ["customers", "orders"]:
        print(f"✓ {view.name} - {view.tableType}")

# 2. Porównaj wyniki PySpark vs SQL (should be identical)
pyspark_count = df_customers.filter(F.col("customer_segment") == "Premium").count()
sql_count = spark.sql("SELECT COUNT(*) FROM customers WHERE customer_segment = 'Premium'").collect()[0][0]

print(f"\n2. PySpark vs SQL consistency:")
print(f"   PySpark filter count: {pyspark_count}")
print(f"   SQL WHERE count: {sql_count}")
print(f"   Match: {'✓' if pyspark_count == sql_count else '✗'}")

# 3. Sprawdź agregacje
agg_result = df_customers.agg(
    F.count("*").alias("total"),
    F.countDistinct("customer_segment").alias("segments"),
    F.countDistinct("country").alias("countries")
).collect()[0]

print(f"\n3. Agregacje:")
print(f"   Total customers: {agg_result['total']}")
print(f"   Unique segments: {agg_result['segments']}")
print(f"   Unique countries: {agg_result['countries']}")

# 4. Sprawdź transformation chains
chained_result = df_customers \
    .filter(F.col("country") == "USA") \
    .withColumn("tier", 
        F.when(F.col("customer_segment") == "Premium", "VIP")
         .otherwise("Standard")
    ) \
    .groupBy("tier") \
    .agg(F.count("*").alias("count")) \
    .orderBy("tier")

print(f"\n4. Transformation chain (USA customers by tier):")
chained_result.show()

# Testy asercyjne
try:
    assert pyspark_count == sql_count, "PySpark i SQL dają różne wyniki!"
    assert agg_result['total'] > 0, "Brak danych w DataFrame!"
    print("\n✓ Wszystkie testy przeszły pomyślnie!")
except AssertionError as e:
    print(f"\n✗ Test failed: {e}")

### Porównanie PySpark vs SQL consistency

Sprawdzenie czy PySpark i SQL dają identyczne wyniki:

In [0]:
# 2. Porównaj wyniki PySpark vs SQL (should be identical)
pyspark_count = df_customers.filter(F.col("customer_segment") == "Premium").count()
sql_count = spark.sql("SELECT COUNT(*) FROM customers WHERE customer_segment = 'Premium'").collect()[0][0]

print(f"PySpark filter count: {pyspark_count}")
print(f"SQL WHERE count: {sql_count}")
print(f"Match: {'✓' if pyspark_count == sql_count else '✗'}")

### Sprawdzenie agregacji

Podstawowe statystyki DataFrame:

In [0]:
# 3. Sprawdź agregacje
agg_result = df_customers.agg(
    F.count("*").alias("total"),
    F.countDistinct("customer_segment").alias("segments"),
    F.countDistinct("country").alias("countries")
).collect()[0]

print(f"Total customers: {agg_result['total']}")
print(f"Unique segments: {agg_result['segments']}")
print(f"Unique countries: {agg_result['countries']}")

### Transformation chain test

Test łańcuchowania transformacji (USA customers by tier):

In [0]:
# 4. Sprawdź transformation chains
chained_result = df_customers \
    .filter(F.col("country") == "USA") \
    .withColumn("tier", 
        F.when(F.col("customer_segment") == "Premium", "VIP")
         .otherwise("Standard")
    ) \
    .groupBy("tier") \
    .agg(F.count("*").alias("count")) \
    .orderBy("tier")

chained_result.show()

# Testy asercyjne
try:
    assert pyspark_count == sql_count, "PySpark i SQL dają różne wyniki!"
    assert agg_result['total'] > 0, "Brak danych w DataFrame!"
    print("\n✓ Wszystkie testy przeszły pomyślnie!")
except AssertionError as e:
    print(f"\n✗ Test failed: {e}")

## Sekcja 6: JOIN - Łączenie danych z wielu źródeł

**Cel sekcji:** Nauka łączenia DataFrames (JOIN) w PySpark i Spark SQL.

**Typy JOIN:**

| Typ | Opis | Przypadek użycia |
|-----|------|------------------|
| **INNER** | Tylko dopasowane rekordy | Standardowe łączenie tabel |
| **LEFT (OUTER)** | Wszystko z lewej + dopasowane z prawej | Zachowanie wszystkich rekordów głównej tabeli |
| **RIGHT (OUTER)** | Wszystko z prawej + dopasowane z lewej | Rzadko używane |
| **FULL (OUTER)** | Wszystkie rekordy z obu stron | Wykrywanie niezgodności |
| **CROSS** | Iloczyn kartezjański | Generowanie kombinacji |
| **LEFT ANTI** | Rekordy z lewej bez dopasowania | "NOT IN" - znajdź brakujące |
| **LEFT SEMI** | Rekordy z lewej z dopasowaniem | "EXISTS" - sprawdź istnienie |

### Przykład 6.1: Przygotowanie danych do JOIN

Najpierw wczytamy dodatkowe dane (orders), żeby mieć co łączyć z customers:

In [ ]:
# Wczytaj orders (JSON)
ORDERS_JSON = f"{DATASET_BASE_PATH}/orders/orders_batch.json"
df_orders = spark.read.json(ORDERS_JSON)

# Zarejestruj jako temp view
df_orders.createOrReplaceTempView("orders")

print(f"✓ Orders: {df_orders.count()} rekordów")
print(f"✓ Customers: {df_customers.count()} rekordów")
print(f"\nOrders schema:")
df_orders.printSchema()

### Przykład 6.2: INNER JOIN - PySpark vs SQL

In [ ]:
# INNER JOIN - PySpark DataFrame API
# Zamówienia z danymi klientów

# Sposób 1: Join po wspólnej nazwie kolumny
inner_join_pyspark = df_orders.join(
    df_customers,
    "customer_id",  # Wspólna kolumna
    "inner"
).select(
    df_orders.order_id,
    df_orders.customer_id,
    df_customers.first_name,
    df_customers.last_name,
    df_customers.customer_segment,
    df_orders.total_amount
)

print("INNER JOIN (PySpark):")
inner_join_pyspark.show(5)

**To samo w Spark SQL:**

In [ ]:
# INNER JOIN - Spark SQL
inner_join_sql = spark.sql("""
    SELECT 
        o.order_id,
        o.customer_id,
        c.first_name,
        c.last_name,
        c.customer_segment,
        o.total_amount
    FROM orders o
    INNER JOIN customers c ON o.customer_id = c.customer_id
    LIMIT 5
""")

print("INNER JOIN (SQL):")
inner_join_sql.show()

### Przykład 6.3: LEFT JOIN - zachowanie wszystkich klientów

In [ ]:
# LEFT JOIN - wszystkich klientów, nawet bez zamówień
# PySpark
left_join = df_customers.join(
    df_orders,
    "customer_id",
    "left"  # lub "left_outer"
).select(
    df_customers.customer_id,
    df_customers.first_name,
    df_customers.customer_segment,
    df_orders.order_id,
    df_orders.total_amount
)

print("LEFT JOIN - wszyscy klienci (z zamówieniami lub bez):")
left_join.show(10)

**Sprawdź klientów BEZ zamówień (NULL w order_id):**

In [ ]:
# Znajdź klientów bez zamówień
customers_without_orders = left_join.filter(F.col("order_id").isNull())

print(f"Klienci bez zamówień: {customers_without_orders.count()}")
customers_without_orders.show(5)

### Przykład 6.4: LEFT ANTI JOIN - "NOT IN" pattern

LEFT ANTI zwraca rekordy z lewej tabeli, które NIE mają dopasowania w prawej:

In [ ]:
# LEFT ANTI JOIN - klienci którzy NIE mają zamówień
# PySpark
anti_join = df_customers.join(
    df_orders,
    "customer_id",
    "left_anti"  # Tylko rekordy z lewej BEZ dopasowania w prawej
)

print(f"Klienci BEZ zamówień (LEFT ANTI): {anti_join.count()}")
anti_join.select("customer_id", "first_name", "last_name", "customer_segment").show(5)

**To samo w SQL (NOT EXISTS):**

In [ ]:
# SQL equivalent - LEFT ANTI jako NOT EXISTS
anti_join_sql = spark.sql("""
    SELECT c.customer_id, c.first_name, c.last_name, c.customer_segment
    FROM customers c
    WHERE NOT EXISTS (
        SELECT 1 FROM orders o WHERE o.customer_id = c.customer_id
    )
""")

print(f"Klienci BEZ zamówień (SQL NOT EXISTS): {anti_join_sql.count()}")
anti_join_sql.show(5)

### Przykład 6.5: LEFT SEMI JOIN - "EXISTS" pattern

LEFT SEMI zwraca rekordy z lewej tabeli, które MAJĄ dopasowanie w prawej (ale bez kolumn z prawej):

In [ ]:
# LEFT SEMI JOIN - klienci którzy MAJĄ zamówienia
# Zwraca tylko kolumny z lewej tabeli (customers)
semi_join = df_customers.join(
    df_orders,
    "customer_id",
    "left_semi"  # Tylko rekordy z lewej Z dopasowaniem w prawej
)

print(f"Klienci Z zamówieniami (LEFT SEMI): {semi_join.count()}")
semi_join.select("customer_id", "first_name", "last_name", "customer_segment").show(5)

### Przykład 6.6: JOIN z agregacją - analiza biznesowa

In [ ]:
# Analiza: Suma zamówień per segment klienta
# Łączymy JOIN z agregacją

# PySpark
segment_analysis = df_orders.join(
    df_customers,
    "customer_id",
    "inner"
).groupBy(
    df_customers.customer_segment
).agg(
    F.count("order_id").alias("order_count"),
    F.sum("total_amount").alias("total_revenue"),
    F.avg("total_amount").alias("avg_order_value"),
    F.countDistinct("customer_id").alias("unique_customers")
).orderBy(F.desc("total_revenue"))

print("Analiza zamówień per segment klienta:")
segment_analysis.show()

**To samo w SQL:**

In [ ]:
# SQL version
segment_analysis_sql = spark.sql("""
    SELECT 
        c.customer_segment,
        COUNT(o.order_id) as order_count,
        SUM(o.total_amount) as total_revenue,
        AVG(o.total_amount) as avg_order_value,
        COUNT(DISTINCT o.customer_id) as unique_customers
    FROM orders o
    INNER JOIN customers c ON o.customer_id = c.customer_id
    GROUP BY c.customer_segment
    ORDER BY total_revenue DESC
""")

print("Analiza zamówień per segment (SQL):")
segment_analysis_sql.show()

### Przykład 6.7: Warunki JOIN - różne kolumny

Gdy nazwy kolumn są różne w obu tabelach:

In [ ]:
# Gdy kolumny mają RÓŻNE nazwy - użyj wyrażenia
# Przykład: gdyby orders miał "cust_id" zamiast "customer_id"

# Sposób 1: Wyrażenie warunkowe
join_different_cols = df_orders.join(
    df_customers,
    df_orders.customer_id == df_customers.customer_id,  # Jawne porównanie
    "inner"
)

# Sposób 2: Lista kolumn (gdy nazwy identyczne)
join_same_cols = df_orders.join(
    df_customers,
    ["customer_id"],  # Lista wspólnych kolumn
    "inner"
)

# Sposób 3: Wiele warunków JOIN
join_multi_condition = df_orders.join(
    df_customers,
    (df_orders.customer_id == df_customers.customer_id) & 
    (df_orders.total_amount > 0),  # Dodatkowy warunek
    "inner"
)

print(f"Join z wyrażeniem: {join_different_cols.count()} rekordów")
print(f"Join z listą kolumn: {join_same_cols.count()} rekordów")
print(f"Join z wieloma warunkami: {join_multi_condition.count()} rekordów")

### 📊 Podsumowanie JOIN

| Typ JOIN | PySpark | SQL | Przypadek użycia |
|----------|---------|-----|------------------|
| **INNER** | `df1.join(df2, "key", "inner")` | `FROM t1 JOIN t2 ON ...` | Tylko dopasowane |
| **LEFT** | `df1.join(df2, "key", "left")` | `FROM t1 LEFT JOIN t2 ON ...` | Wszystko z lewej |
| **LEFT ANTI** | `df1.join(df2, "key", "left_anti")` | `WHERE NOT EXISTS` | Brakujące rekordy |
| **LEFT SEMI** | `df1.join(df2, "key", "left_semi")` | `WHERE EXISTS` | Sprawdzenie istnienia |
| **CROSS** | `df1.crossJoin(df2)` | `FROM t1 CROSS JOIN t2` | Iloczyn kartezjański |

> **💡 Tip:** Dla optymalizacji JOIN z małą tabelą użyj `broadcast()`:
> ```python
> from pyspark.sql.functions import broadcast
> df_large.join(broadcast(df_small), "key")
> ```

## Troubleshooting

### Problem 1: AnalysisException - kolumna nie istnieje
**Objawy:**
- `AnalysisException: cannot resolve 'column_name'`
- Błąd przy select() lub withColumn()

**Rozwiązanie:**
```python
# Sprawdź dostępne kolumny
print(df.columns)

# Sprawdź schemat
df.printSchema()

# Case sensitivity - Spark SQL jest case-insensitive, ale DataFrame API może być
df.select(F.col("`Column Name`"))  # Użyj backticks dla nazw z spacjami
```

### Problem 2: TypeError przy multiple conditions
**Objawy:**
- `TypeError: unsupported operand type(s) for &`
- Błąd przy łączeniu warunków

**Rozwiązanie:**
```python
# BŁĄD: Brak nawiasów wokół warunków
df.filter(F.col("age") > 18 & F.col("country") == "Poland")

# POPRAWNIE: Nawiasy wokół każdego warunku
df.filter((F.col("age") > 18) & (F.col("country") == "Poland"))

# Alternatywnie: Użyj metod
df.filter(F.col("age") > 18).filter(F.col("country") == "Poland")
```

### Problem 3: Aggregation without groupBy
**Objawy:**
- Błąd przy próbie agregacji bez grupowania
- Unexpected results

**Rozwiązanie:**
```python
# Agregacja całego DataFrame (bez groupBy)
df.agg(F.sum("amount"), F.avg("age"))

# Agregacja per grupa (z groupBy)
df.groupBy("country").agg(F.sum("amount"), F.avg("age"))

# BŁĄD: agg() bez groupBy gdy chcesz per grupa
# df.agg(F.sum("amount")).groupBy("country")  # Źle!
```

### Problem 4: SQL view not found
**Objawy:**
- `Table or view not found: view_name`
- SQL query fails

**Rozwiązanie:**
```python
# Sprawdź dostępne views
spark.catalog.listTables()

# Utwórz view jeśli nie istnieje
df.createOrReplaceTempView("view_name")

# Global temp view (dostępny w całej aplikacji)
df.createOrReplaceGlobalTempView("view_name")
# Użycie: spark.sql("SELECT * FROM global_temp.view_name")
```

### Debugging tips:
- Użyj `.explain()` aby zobaczyć execution plan i zidentyfikować bottlenecks
- `.show(truncate=False)` dla pełnych wartości kolumn
- `.printSchema()` dla sprawdzenia typów danych
- `df.columns` dla listy kolumn
- `.count()` jest action - triggeruje execution (użyj ostrożnie w loops)
- Cache DataFrame jeśli używasz wielokrotnie: `df.cache()`

## Best Practices

### Wydajność:
- Używaj column pruning (select tylko potrzebne kolumny) aby zmniejszyć shuffle data
- Filter pushdown - filtruj jak najwcześniej w pipeline
- Predicate pushdown - Spark automatycznie przesuwa filtry do źródła danych
- Cache DataFrames używane wielokrotnie: `df.cache()` lub `df.persist()`
- Unikaj `collect()` na dużych DataFrames (OOM risk)
- Używaj `broadcast()` dla małych DataFrames w joinach

### Jakość kodu:
- Nazywaj kolumny czytelnie używając `.alias()`
- Komentuj złożoną logikę biznesową
- Łańcuchuj transformacje czytelnie (jedna transformacja per linię)
- Używaj zmiennych dla czytelności zamiast głębokich nested expressions
- Konsystencja: Wybierz PySpark LUB SQL dla całego pipeline'u (nie mieszaj bez potrzeby)
- Type hints w Python dla DataFrame-returning functions

### Transformacje:
- Preferuj `filter()` przed `groupBy()` (pre-aggregation filtering)
- `when().otherwise()` jest lazy evaluated - bezpieczne dla null values
- `withColumn()` nadpisuje kolumnę jeśli już istnieje (użyj świadomie)
- Unikaj UDFs (User Defined Functions) - wolniejsze niż built-in functions
- Używaj built-in functions (F.*) zamiast własnych - zoptymalizowane przez Catalyst

### SQL vs PySpark:
- SQL dla ad-hoc analiz i eksploracji
- PySpark dla production pipelines i ETL
- Dokumentuj wybór (dlaczego SQL vs PySpark w danym przypadku)
- Testuj oba podejścia dla złożonych zapytań (readability vs maintainability)

### Governance:
- Temporary views są session-scoped - nie używaj dla shared data
- Persistent views (CREATE VIEW) dla reusable logic
- Dokumentuj business logic w komentarzach i docstrings
- Version control dla notebooks (Git integration)

## Podsumowanie

### Co zostało osiągnięte:
- Załadowanie danych z Unity Catalog Volume (customers.csv, orders.json)
- Transformacje kolumnowe (select, withColumn, drop, alias)
- Logika warunkowa (when/otherwise, CASE WHEN)
- Operacje tekstowe (trim, upper, lower, regex, split)
- Filtry i sortowanie (filter, where, orderBy, multiple conditions)
- Agregacje (groupBy, agg, rollup, cube)
- Porównanie PySpark DataFrame API vs SQL (identyczna wydajność)
- Zrozumienie Catalyst Optimizer i lazy evaluation

### Kluczowe wnioski:
1. **PySpark i SQL są równoważne**: Catalyst Optimizer kompiluje oba do tego samego execution plan
2. **Lazy evaluation**: Transformacje budują DAG, execution następuje przy action
3. **Column pruning i filter pushdown**: Spark automatycznie optymalizuje queries
4. **Wybór narzędzia**: Zależy od use case, zespołu i czytelności (nie od wydajności)

### Quick Reference - Najważniejsze komendy:

| Operacja | PySpark | SQL |
|----------|---------|-----|
| Selekcja kolumn | `df.select("col1", "col2")` | `SELECT col1, col2 FROM table` |
| Nowa kolumna | `df.withColumn("new", expr)` | `SELECT *, expr AS new FROM table` |
| Filtr | `df.filter(F.col("age") > 18)` | `SELECT * FROM table WHERE age > 18` |
| Sortowanie | `df.orderBy(F.desc("amount"))` | `SELECT * FROM table ORDER BY amount DESC` |
| Agregacja | `df.groupBy("country").agg(F.sum("sales"))` | `SELECT country, SUM(sales) FROM table GROUP BY country` |
| Logika warunkowa | `F.when(cond, val).otherwise(default)` | `CASE WHEN cond THEN val ELSE default END` |
| Operacje tekstowe | `F.upper(F.trim(F.col("name")))` | `UPPER(TRIM(name))` |
| Join | `df1.join(df2, "key")` | `SELECT * FROM t1 JOIN t2 ON t1.key = t2.key` |

### Następne kroki:
- **Kolejny notebook**: 04_data_cleaning_quality.ipynb - Data quality i czyszczenie danych
- **Warsztat praktyczny**: 02_transformations_cleaning_workshop.ipynb
- **Materiały dodatkowe**: 
  - [Spark SQL Guide](https://spark.apache.org/docs/latest/sql-programming-guide.html)
  - [PySpark API Reference](https://spark.apache.org/docs/latest/api/python/)
- **Zadanie domowe**: Wykonaj top 10 customers per country używając zarówno PySpark jak i SQL

## Czyszczenie zasobów

Posprzątaj zasoby utworzone podczas notebooka:

In [0]:
# Opcjonalne czyszczenie zasobów testowych
# UWAGA: Uruchom tylko jeśli chcesz usunąć wszystkie utworzone dane

# Usuń temporary views
spark.catalog.dropTempView("customers")
spark.catalog.dropTempView("orders")

# Wyczyść cache
spark.catalog.clearCache()

**Czyszczenie zakończone:**

- Temporary views `customers` i `orders` zostały usunięte
- Cache został wyczyszczony  
- Dane źródłowe w Volume pozostają nienaruszone